In [1]:
import pickle
import pandas as pd
import numpy as np
from collections import defaultdict
import time, os
import re

## Downloading and casting to dataframe

In [93]:
with open('../videos_info.pkl', 'rb') as picklefile:
    videos_info = pickle.load(picklefile)
with open('../channels_about_607_att2.pkl', 'rb') as picklefile:
    channels_about = pickle.load(picklefile)
with open('../channels_videos_page.pkl', 'rb') as picklefile:
    channels_videos_page = pickle.load(picklefile)

In [94]:
channels_about_df = pd.DataFrame.from_dict(channels_about).T
channels_videos_page_df = pd.DataFrame.from_dict(channels_videos_page).T
videos_info_df = pd.DataFrame.from_dict(videos_info).T

## Cleaning channels_videos_page_df

In [95]:
channels_videos_page_df.head(3)

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
https://www.youtube.com/user/yogawithadriene,{'title': 'Yoga for Vulnerability | Move #wi...,{'title': 'Awaken The Artist Within | Yoga W...,{'title': 'Yoga Kiss | Total Body Flow | Yoga...,{'title': 'Home - Day 30 - Journey | 30 Days...,{'title': 'Home - Day 29 - Intuit | 30 Days ...,{'title': 'Home - Day 28 - Nourish | 30 Days...,{'title': 'Home - Day 27 - Integrate | 30 Da...,{'title': 'Home - Day 26 - Keep | 30 Days of...,{'title': 'Home - Day 25 - Unwind | 30 Days ...,{'title': 'Home - Day 24 - Uplift | 30 Days ...,...,{'title': 'Home - Day 13 - Grow | 30 Days of ...,{'title': 'Home - Day 12 - Nurture | 30 Days...,{'title': 'Home - Day 11 - Dig | 30 Days of ...,{'title': 'Home - Day 10 - Ground | 30 Days ...,{'title': 'Home - Day 9 - Inquire | 30 Days ...,{'title': 'Home - Day 8 - Heal | 30 Days of ...,{'title': 'Home - Day 7 - Stretch | 30 Days ...,{'title': 'Home - Day 6 - Ignite | 30 Days o...,{'title': 'Home - Day 5 - Soften | 30 Days o...,{'title': 'Home - Day 4 - Activate | 30 Days...
https://www.youtube.com/channel/UCahoXO8yh6mqJD9EiARHI7w,{'title': '30 Min Yoga Flow: Cleanse Your Sacr...,{'title': '30 Min Yoga Flow: Cleanse Your Root...,{'title': '30 Minute Full Body Power Yoga Prac...,{'title': '15 Minute Guided Body Scan Meditati...,{'title': '20 Min Morning Yoga Stretch to Ener...,{'title': '20 Min Morning Yoga: Start Your Day...,{'title': 'Yoga for Soothing Sleep: 20 Min Bed...,{'title': '15 Minute Meditation: Guided Medita...,{'title': '30 Min Full Body Vinyasa Yoga: Crow...,{'title': '45 Min Full Body Vinyasa Yoga: Root...,...,{'title': 'WE LOST OUR CREW 😔Last Hurrah in Gr...,{'title': 'Chakra Chats: Root Chakra Explained...,{'title': 'Best Bedtime Yoga: 10 Minute Routin...,{'title': 'Chakra Chats: Intro to the Chakra S...,{'title': 'Our first big sail and we ALMOST SA...,{'title': 'Power Yoga Class: 20 Minutes to Awa...,"{'title': 'AFTER 3 MONTHS, we get our stuff fr...",{'title': 'Gentle Yoga for Low Back Relief: 15...,"{'title': 'Goodbye First World, Hello Caribbea...",{'title': 'Best Morning Yoga: Full-Body Class ...
https://www.youtube.com/channel/UCLevFC-tyOhyyuhXyTMHJbQ,{'title': 'Hip Mobility Class l Lizette Pompa ...,{'title': 'Shoulder Mobility - From Flexibilit...,{'title': 'Pistol Squat Tutorial - Pistol Squa...,"{'title': 'Yoga Dinámico - Clase de Tadasana',...",{'title': 'Lizette Pompa Yoga at Hot Yoga Upps...,"{'title': 'Dynamic Yoga - 25 Minutes', 'views_...",{'title': 'Yoga for Beginner Classes | 20 Min ...,"{'title': '1 Minute Workout Exercises Video', ...",{'title': 'Yoga for beginners - Wrist Warm Up ...,{'title': 'From side to side - Vinyasa Yoga fl...,...,{'title': '40 minutes vinyasa flow yoga class ...,{'title': 'Monday Yoga workouts with Lizette #...,{'title': 'crow pose yoga tutorial | bakasana ...,"{'title': 'Shoulders on the go', 'views_days':...",{'title': 'Chaturanga Dandasana for Beginners ...,{'title': 'Beginner yoga - Spring flow - Begin...,{'title': '15-Min Vinyasa Flow Class | Yoga Ch...,{'title': 'Moving it - Vinyasa flow class | Yo...,{'title': 'Monday Yoga workout with Lizette #9...,{'title': 'Full body on the go - Vinyasa class...


In [96]:
channels_videos_page_df = channels_videos_page_df.reset_index()

In [97]:
videos_scraped = channels_videos_page_df.columns[1:]

In [98]:
channels_videos_page_df = pd.melt(channels_videos_page_df, id_vars='index', value_vars = videos_scraped)

In [99]:
splitted_value_col = channels_videos_page_df['value'].apply(pd.Series)
channels_videos_page_df = pd.concat([channels_videos_page_df, splitted_value_col], axis= 1)

In [100]:
channels_videos_page_df.columns

Index(['index', 'variable', 'value', 0, 'duration', 'title', 'views_days'], dtype='object')

In [101]:
channels_videos_page_df = channels_videos_page_df[['index', 'variable', 'duration', 'title', 'views_days']]
channels_videos_page_df.rename(columns = {'index':'channel_URL', 'variable':'video_order_num'}, inplace=True)

In [102]:
channels_videos_page_df.shape

(17640, 5)


Cleaning: Duration

In [103]:
channels_videos_page_df['duration'].isna().sum()

2984

NaNs is a result of video tab having no content: https://www.youtube.com/user/yogatoday, https://www.youtube.com/user/yogaworks
Here function didn't work: https://www.youtube.com/user/divinedesignwellness

In [104]:
channels_videos_page_df[channels_videos_page_df['duration'].isna()].sample(6)

,channel_URL,video_order_num,duration,title,views_days
2530,https://www.youtube.com/user/SiyaYoga,5,NaN,NaN,NaN
8559,https://www.youtube.com/channel/UC4gag6NoyUl0I...,15,NaN,NaN,NaN
13205,https://www.youtube.com/channel/UC9L2KRYM9a5g1...,23,NaN,NaN,NaN
3289,https://www.youtube.com/channel/UCZqajOFub53l1...,6,NaN,NaN,NaN
11937,https://www.youtube.com/channel/UCw6H36jMapss7...,21,NaN,NaN,NaN
17016,https://www.youtube.com/channel/UCEl4J6Zg3tlPu...,29,NaN,NaN,NaN


In [105]:
channels_videos_page_df = channels_videos_page_df[channels_videos_page_df['duration'].notna()]

In [106]:
channels_videos_page_df['duration'] = channels_videos_page_df['duration'].str.replace('- Duration:','').str.replace('.','')

In [107]:
channels_videos_page_df['duration'] = channels_videos_page_df['duration'].str.replace(' minutes', 'M').str.replace(' seconds', 'S').str.replace(' hours', 'H').str.replace(' minute', 'M').str.replace(' second', 'S').str.replace(' hour', 'H').str.replace(',', '').str.replace(' ', '') 

In [108]:
def parse_duration(string):
    try:
        dur = pd.to_datetime(string, format = '%MM%SS') 
    except:
        try:
            dur = pd.to_datetime(string, format = '%HH%MM')
        except:
            try:
                dur = pd.to_datetime(string, format = '%HH%MM%SS')
            except:
                try:
                    dur = pd.to_datetime(string, format = '%HH')
                except:
                    try:
                        dur = pd.to_datetime(string, format = '%MM')
                    except:
                        try:
                            if int(string.replace('S','')) > 60:
                                string = '60S'  #videos with duration a little over 1 min display sec > 60
                            dur = pd.to_datetime(string, format = '%SS')
                        except:
                            dur = ''
    return dur

In [109]:
channels_videos_page_df['duration'] = channels_videos_page_df['duration'].apply(parse_duration)

In [110]:
channels_videos_page_df['duration'] = (channels_videos_page_df['duration'].dt.hour * 60 + channels_videos_page_df['duration'].dt.minute) * 60 + channels_videos_page_df['duration'].dt.second

Cleaning: Views & Days

In [111]:
channels_videos_page_df['views_days'].isna().sum()

0

In [112]:
channels_videos_page_df['views_days'] = channels_videos_page_df['views_days'].str.replace('No', '0')
channels_videos_page_df['views_days'] = channels_videos_page_df['views_days'].str.replace('views', 'view')

In [113]:
channels_videos_page_df['views_video'] = channels_videos_page_df['views_days'].apply(lambda string: (re.sub('[^0-9]+', '', string)))

In [114]:
channels_videos_page_df['views_video']=pd.to_numeric(channels_videos_page_df['views_video'], errors = 'coerce')

In [115]:
channels_videos_page_df['video_age'] = channels_videos_page_df['views_days'].apply(lambda string: re.sub('([0-9]+,[0-9]+ view)|([0-9]+ view)', '', string).replace(' ago', ''))

In [116]:
channels_videos_page_df['video_age'] = channels_videos_page_df['video_age'].apply(lambda string: (re.sub('s', '', string)))
channels_videos_page_df['video_age'] = channels_videos_page_df['video_age'].apply(lambda string: (re.sub(',', '.', string)))

In [117]:
channels_videos_page_df['video_age'].unique()

array(['4 day', '1 day', '7 month', '6 year', '3 day', '14 hour',
       '2 year', '5 year', '12 hour', '2 week', '1 year', '3 week',
       '1 hour', '5 day', '1 week', '4 hour', '9 month', '2 day',
       '2 month', '23 hour', '1 month', '8 month', '3 year', '6 day',
       '32 minute', '11 hour', '3 month', 'Start:', '4 year', '3 hour',
       '1.6 year', '19 hour', '6 hour', '6 month', '22 hour', '9 minute',
       '5 month', '2 hour', '8 hour', '7 hour', '4 week', '18 hour',
       '10 hour', '9 hour', '56 minute', '26 minute', '10 month',
       '16 hour', '7 minute', '4 month', '15 hour', '11 month',
       '28 minute', '20 hour', '5 hour', '21 hour', '17 hour', '1.4 year',
       '3.2 year', '5.1 year', '11.3 year', '13 hour', '39 minute',
       '22.1 year', '8.3 year', '7 year', '26.1 year', '1.2 month',
       '1.3 year', '27.5 year', '6.3 year', '2.1 year', '1.2 year',
       '6.1 year', '15.3 year', '13.3 year', '8 year', '4.6 year',
       '8.4 year', '1.8 year', '3.3 yea

In [118]:
def video_age_to_min(age):
    if 'minute' in age:
        video_age = float(age.replace('minute',''))
    elif 'hour' in age:
        video_age = float(age.replace('hour','')) * 60
    elif 'day' in age:
        video_age = float(age.replace('day','')) * 60 * 24
    elif 'month' in age:
        video_age = float(age.replace('month','')) * 60 * 24 * 30
    elif 'year' in age:
        video_age = float(age.replace('year','')) * 60 * 24 * 30 * 365
    else:
        video_age = 0.0
    return video_age

In [119]:
channels_videos_page_df['video_age_min'] = channels_videos_page_df['video_age'].apply(video_age_to_min)

Miscellaneous

In [120]:
channels_videos_page_df['video_order_num']=pd.to_numeric(channels_videos_page_df['video_order_num'], errors = 'coerce')

In [121]:
channels_videos_page_df['video_age_min'].isna().sum()

0

In [122]:
channels_videos_page_df = channels_videos_page_df[['channel_URL', 
                                                   'video_order_num', 
                                                   'duration', 
                                                   'title', 
                                                   'views_video',
                                                   'video_age_min']]

In [123]:
channels_videos_page_df.shape

(14656, 6)

In [124]:
channels_videos_page_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14656 entries, 0 to 17639
Data columns (total 6 columns):
channel_URL        14656 non-null object
video_order_num    14656 non-null int64
duration           14656 non-null int64
title              14656 non-null object
views_video        14611 non-null float64
video_age_min      14656 non-null float64
dtypes: float64(2), int64(2), object(2)
memory usage: 801.5+ KB


In [125]:
#NaNs are premieres, remove those.
channels_videos_page_df[channels_videos_page_df['views_video'].isna()].head()

,channel_URL,video_order_num,duration,title,views_video,video_age_min
78,https://www.youtube.com/channel/UCMOuxZneKXsKi...,1,325,Yoga Tutorial For Beginners - Supine Twist wit...,NaN,0.0
160,https://www.youtube.com/user/yogavedanta,1,4620,"Action, renunciation and indifference to objec...",NaN,0.0
171,https://www.youtube.com/user/AustSchoolMeditation,1,1620,Energising Morning Meditation with Vrindavan d...,NaN,0.0
465,https://www.youtube.com/user/YogaVidyaVortrag,1,411,Shankaracharyas Lehren - Sukadev | Vedanta | M...,NaN,0.0
748,https://www.youtube.com/user/yogavedanta,2,5160,Devotion and dharma - James Swartz - Yoga of L...,NaN,0.0


In [126]:
channels_videos_page_df = channels_videos_page_df[channels_videos_page_df['views_video'].notna()]

In [127]:
channels_videos_page_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14611 entries, 0 to 17639
Data columns (total 6 columns):
channel_URL        14611 non-null object
video_order_num    14611 non-null int64
duration           14611 non-null int64
title              14611 non-null object
views_video        14611 non-null float64
video_age_min      14611 non-null float64
dtypes: float64(2), int64(2), object(2)
memory usage: 799.0+ KB


In [128]:
channels_videos_page_df.head(3)

,channel_URL,video_order_num,duration,title,views_video,video_age_min
0,https://www.youtube.com/user/yogawithadriene,1,2100,Yoga for Vulnerability | Move #withme | Yo...,3440914.0,5760.0
1,https://www.youtube.com/channel/UCahoXO8yh6mqJ...,1,2040,30 Min Yoga Flow: Cleanse Your Sacral Chakra |...,25491.0,1440.0
2,https://www.youtube.com/channel/UCLevFC-tyOhyy...,1,761,Hip Mobility Class l Lizette Pompa Yoga,29437.0,302400.0


## Cleaning channels_about_df

In [129]:
channels_about_df.head(3)

,name,total_views,joined_date,subscriber_count,n_featured_channels,n_links,description_length,description
https://www.youtube.com/user/yogawithadriene,YogaWithAdriene.com,"563,444,717","Joined Aug 30, 2012",6.87M,3,10,946,WELCOME to Yoga With Adriene! Our mission is t...
https://www.youtube.com/channel/UCTlmEOYH5tLJQP42gn1CkbA,Patreon,"12,695,198","Joined Dec 12, 2016",122K,2,10,13,Vlog channel.
https://www.youtube.com/channel/UCWuSqOUb_WzHoY9tn7CeMXQ,,"121,125","Joined Dec 13, 2017",6.75K,0,0,343,Power Life is about making your healthy lifest...


In [130]:
channels_about_df = channels_about_df.reset_index()

In [131]:
channels_about_df.columns

Index(['index', 'name', 'total_views', 'joined_date', 'subscriber_count',
       'n_featured_channels', 'n_links', 'description_length', 'description'],
      dtype='object')

In [132]:
channels_about_df.rename(columns = {'index':'channel_URL', 
                                          'name':'channel_name', 
                                          'total_views': 'total_views_channel', 
                                          'joined_date':'channel_created_date', 
                                          'n_links':'n_promo_links', 
                                          'description_length':'channel_desc_length', 
                                          'description': 'channel_desc'}, inplace=True)

Cleaning Subscriber Count

In [133]:
channels_about_df['subscriber_count'].isna().sum()

0

In [134]:
def strval_to_float(subscriber_count):
    if type(subscriber_count) == float or type(subscriber_count) == int:
        sc = subscriber_count
    elif 'K' in subscriber_count:
        sc = float(subscriber_count.replace('K',''))*1000
    elif 'M' in subscriber_count:
        sc = float(subscriber_count.replace('M',''))*1000000
    else:
        sc = 0.0
    return sc

In [135]:
channels_about_df['subscriber_count'] = channels_about_df['subscriber_count'].apply(lambda string: (strval_to_float(string)))

Cleaning Total Views by Channel

In [136]:
channels_about_df['total_views_channel'].isna().sum()

0

In [137]:
channels_about_df['total_views_channel'] = channels_about_df['total_views_channel'].str.replace(',', '')

In [138]:
channels_about_df['total_views_channel'] = pd.to_numeric(channels_about_df['total_views_channel'], errors = 'coerce')

Cleaning Channel Created Date

In [139]:
channels_about_df['channel_created_date'].isna().sum()

0

In [140]:
channels_about_df['channel_created_date'] = channels_about_df['channel_created_date'].str.replace('Joined ', '')

In [141]:
channels_about_df['channel_created_date'] = pd.to_datetime(channels_about_df['channel_created_date'])

Miscellaneous

In [142]:
channels_about_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 9 columns):
channel_URL             607 non-null object
channel_name            607 non-null object
total_views_channel     595 non-null float64
channel_created_date    595 non-null datetime64[ns]
subscriber_count        607 non-null float64
n_featured_channels     607 non-null object
n_promo_links           607 non-null object
channel_desc_length     607 non-null object
channel_desc            607 non-null object
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 42.8+ KB


In [143]:
channels_about_df['channel_desc_length']=pd.to_numeric(channels_about_df['channel_desc_length'], errors = 'coerce')
channels_about_df['n_promo_links']=pd.to_numeric(channels_about_df['n_promo_links'], errors = 'coerce')
channels_about_df['n_featured_channels']=pd.to_numeric(channels_about_df['n_featured_channels'], errors = 'coerce')

In [144]:
channels_about_df.head(3)

,channel_URL,channel_name,total_views_channel,channel_created_date,subscriber_count,n_featured_channels,n_promo_links,channel_desc_length,channel_desc
0,https://www.youtube.com/user/yogawithadriene,YogaWithAdriene.com,563444717.0,2012-08-30,6870000.0,3.0,10.0,946.0,WELCOME to Yoga With Adriene! Our mission is t...
1,https://www.youtube.com/channel/UCTlmEOYH5tLJQ...,Patreon,12695198.0,2016-12-12,122000.0,2.0,10.0,13.0,Vlog channel.
2,https://www.youtube.com/channel/UCWuSqOUb_WzHo...,,121125.0,2017-12-13,6750.0,0.0,0.0,343.0,Power Life is about making your healthy lifest...


In [147]:
channels_about_df = channels_about_df[channels_about_df['total_views_channel'].notna()]

## Cleaning videos_info_df

In [148]:
videos_info_df = pd.DataFrame.from_dict(videos_info).T

In [149]:
videos_info_df.tail(3)

Title  \
https://www.youtube.com/user/VideoJug A0GC4z60TPE  Ben & Jimmy's Guide To London: First Impressions   
                                      8hHj27QazIQ     Ben & Jimmy's Guide To London: Londoners E1/3   
                                      3RifUHkAon0                Tyra Banks Makeup - Models' Corner   

                                                  Views  \
https://www.youtube.com/user/VideoJug A0GC4z60TPE  2015   
                                      8hHj27QazIQ  1534   
                                      3RifUHkAon0  3128   

                                                                                         Description  \
https://www.youtube.com/user/VideoJug A0GC4z60TPE  In honour of YouTube Comedy Week we've launche...   
                                      8hHj27QazIQ  This week is YouTube Comedy Week, so we are br...   
                                      3RifUHkAon0  Tyra Banks always has defined cheekbones and s...   

                                                            Publication Date  \
https://www.youtube.com/user/VideoJug A0GC4z60TPE  Published on May 22, 2013   
                                      8hHj27QazIQ  Published on May 20, 2013   
                                      3RifUHkAon0  Published on May 16, 2013   

                                                  Likes Dislikes  
https://www.youtube.com/user/VideoJug A0GC4z60TPE    20        4  
                                      8hHj27QazIQ    19        4  
                                      3RifUHkAon0    52        4

In [150]:
videos_info_df = videos_info_df.reset_index()

In [151]:
videos_info_df.columns

Index(['level_0', 'level_1', 'Title', 'Views', 'Description',
       'Publication Date', 'Likes', 'Dislikes'],
      dtype='object')

In [152]:
videos_info_df = videos_info_df[['level_0', 'level_1', 'Title', 'Views', 'Description', 'Publication Date', 'Likes', 'Dislikes']]
videos_info_df.rename(columns = {'level_0':'channel_URL', 
                                 'level_1':'video_URL_ext', 
                                 'Title':'title', 
                                 'Views':'views_DUPL', 
                                 'Description':'video_desc', 
                                 'Likes':'likes', 
                                 'Dislikes':'dislikes', 'Publication Date': 'video_published_date'}, inplace=True)

In [153]:
videos_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9399 entries, 0 to 9398
Data columns (total 8 columns):
channel_URL             9399 non-null object
video_URL_ext           9399 non-null object
title                   9399 non-null object
views_DUPL              9399 non-null object
video_desc              9399 non-null object
video_published_date    9399 non-null object
likes                   9399 non-null object
dislikes                9399 non-null object
dtypes: object(8)
memory usage: 587.6+ KB


In [154]:
videos_info_df['video_desc_length'] = videos_info_df['video_desc'].apply(len)

Cleaning Video Published Date

In [155]:
pattern = re.compile("Published|Premiered|Streamed live")
videos_info_df['delivery_method'] = videos_info_df['video_published_date'].apply(lambda string: str(pattern.findall(string)))

In [156]:
videos_info_df['delivery_method'] = videos_info_df['delivery_method'].apply(lambda string: string.replace("'", '').replace('[', '').replace(']', ''))

In [157]:
videos_info_df['video_published_date'] = videos_info_df['video_published_date'].str.replace('.+ ago', 'Streamed live on Apr 9, 2020')

In [158]:
videos_info_df['video_published_date'] = videos_info_df['video_published_date'].str.replace('Published on ', '').str.replace('Streamed live on ', '').str.replace('Premiered ', '')

In [159]:
videos_info_df['video_published_date'] =  pd.to_datetime(videos_info_df['video_published_date'])

In [160]:
videos_info_df['video_published_date'].isna().sum()

0

Miscellaneous

In [161]:
videos_info_df['likes']=pd.to_numeric(videos_info_df['likes'], errors = 'coerce')
videos_info_df['dislikes']=pd.to_numeric(videos_info_df['dislikes'], errors = 'coerce')
videos_info_df['views_DUPL']=pd.to_numeric(videos_info_df['views_DUPL'], errors = 'coerce')

In [162]:
videos_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9399 entries, 0 to 9398
Data columns (total 10 columns):
channel_URL             9399 non-null object
video_URL_ext           9399 non-null object
title                   9399 non-null object
views_DUPL              9399 non-null int64
video_desc              9399 non-null object
video_published_date    9399 non-null datetime64[ns]
likes                   9399 non-null int64
dislikes                9399 non-null int64
video_desc_length       9399 non-null int64
delivery_method         9399 non-null object
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 734.4+ KB


## Merging dataframes

In [163]:
channels_about_df.sample()

,channel_URL,channel_name,total_views_channel,channel_created_date,subscriber_count,n_featured_channels,n_promo_links,channel_desc_length,channel_desc
493,https://www.youtube.com/user/TaraStilesYoga,Tara Stiles,31914332.0,2008-03-11,328000.0,4.0,12.0,864.0,Slow down. Soften. Feel. Respond. Everyone is ...


In [164]:
channels_videos_page_df.sample()

,channel_URL,video_order_num,duration,title,views_video,video_age_min
13343,https://www.youtube.com/channel/UClkJGLnggRrnt...,23,324,New hot yoga Challenge | extreme Yoga Challen...,12852.0,31536000.0


In [165]:
videos_info_df.sample()

,channel_URL,video_URL_ext,title,views_DUPL,video_desc,video_published_date,likes,dislikes,video_desc_length,delivery_method
3207,https://www.youtube.com/channel/UC-enqt3oO6foL...,1xypKS_KRYo,Yoga pour les 7 jours de la semaine: SAMEDI [d...,986,"SI vous pensez que ce cours a une valeur, vous...",2019-10-13,28,0,315,Streamed live


In [166]:
channels_videos_page_df.set_index('title', inplace=True)

In [167]:
videos_info_df.set_index('title', inplace=True)

In [168]:
videos_df = channels_videos_page_df.join(videos_info_df, lsuffix='_page', rsuffix='_info', how='outer')

In [169]:
videos_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15999 entries, "Align Your Yoga, Align Your Life:" a Revolutionary Online Yoga Course to 🦋YOGA EQUINOCCIO PRIMAVERA 🌷🌿Empieza el ciclo! Spring equinox yoga
Data columns (total 14 columns):
channel_URL_page        15735 non-null object
video_order_num         15735 non-null float64
duration                15735 non-null float64
views_video             15735 non-null float64
video_age_min           15735 non-null float64
channel_URL_info        10535 non-null object
video_URL_ext           10535 non-null object
views_DUPL              10535 non-null float64
video_desc              10535 non-null object
video_published_date    10535 non-null datetime64[ns]
likes                   10535 non-null float64
dislikes                10535 non-null float64
video_desc_length       10535 non-null float64
delivery_method         10535 non-null object
dtypes: datetime64[ns](1), float64(8), object(5)
memory usage: 1.8+ MB


In [170]:
videos_df = videos_df[videos_df['likes'].notna()]
videos_df = videos_df[videos_df['video_age_min'].notna()]

In [171]:
videos_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10271 entries, "Align Your Yoga, Align Your Life:" a Revolutionary Online Yoga Course to 🦋YOGA EQUINOCCIO PRIMAVERA 🌷🌿Empieza el ciclo! Spring equinox yoga
Data columns (total 14 columns):
channel_URL_page        10271 non-null object
video_order_num         10271 non-null float64
duration                10271 non-null float64
views_video             10271 non-null float64
video_age_min           10271 non-null float64
channel_URL_info        10271 non-null object
video_URL_ext           10271 non-null object
views_DUPL              10271 non-null float64
video_desc              10271 non-null object
video_published_date    10271 non-null datetime64[ns]
likes                   10271 non-null float64
dislikes                10271 non-null float64
video_desc_length       10271 non-null float64
delivery_method         10271 non-null object
dtypes: datetime64[ns](1), float64(8), object(5)
memory usage: 1.2+ MB


In [172]:
videos_df.head(3)

,channel_URL_page,video_order_num,duration,views_video,video_age_min,channel_URL_info,video_URL_ext,views_DUPL,video_desc,video_published_date,likes,dislikes,video_desc_length,delivery_method
title,,,,,,,,,,,,,,
"""Align Your Yoga, Align Your Life:"" a Revolutionary Online Yoga Course",https://www.youtube.com/user/oceanworshipper,1.0,349.0,932.0,2880.0,https://www.youtube.com/user/oceanworshipper,3GYdWbDE_0g,102.0,"Align Your Yoga, Align Your Life is a deep div...",2020-04-07,10.0,0.0,1008.0,Published
"""Fasting Can Save Your Life"" Students sharing self experience of Therapeutic Fasting",https://www.youtube.com/channel/UC79-TyulXkC7K...,18.0,6060.0,168512.0,86400.0,https://www.youtube.com/channel/UC79-TyulXkC7K...,9zm1_yGBA44,16883.0,Dr. Gulab Rai Tewani is working as CMO (Chief ...,2020-01-25,430.0,45.0,945.0,Published
"""Health Is Wealth"" - Join Detox Camp at SHYNCH, Bhopal",https://www.youtube.com/channel/UC79-TyulXkC7K...,15.0,7140.0,75351.0,43200.0,https://www.youtube.com/channel/UC79-TyulXkC7K...,3jBZkq7eh0Q,7535.0,Sant Hirdaram Yoga & Nature Cure Hospital (Aro...,2020-03-05,217.0,13.0,1605.0,Published


In [173]:
#videos_df[(videos_df['channel_URL_page'] == videos_df['channel_URL_info'])]

In [174]:
videos_df['channel_URL'] = videos_df['channel_URL_info']
videos_df.set_index('channel_URL', inplace=True)
channels_about_df.set_index('channel_URL', inplace=True)

In [175]:
video_channel_df = pd.merge(videos_df, channels_about_df, on='channel_URL', how='outer')

In [176]:
video_channel_df.head()

,channel_URL_page,video_order_num,duration,views_video,video_age_min,channel_URL_info,video_URL_ext,views_DUPL,video_desc,video_published_date,...,video_desc_length,delivery_method,channel_name,total_views_channel,channel_created_date,subscriber_count,n_featured_channels,n_promo_links,channel_desc_length,channel_desc
channel_URL,,,,,,,,,,,,,,,,,,,,,
https://www.youtube.com/user/oceanworshipper,https://www.youtube.com/user/oceanworshipper,1.0,349.0,932.0,2880.0,https://www.youtube.com/user/oceanworshipper,3GYdWbDE_0g,102.0,"Align Your Yoga, Align Your Life is a deep div...",2020-04-07,...,1008.0,Published,Donate to EcoKarma,237924.0,2008-02-26,3070.0,0.0,2.0,288.0,Blissology is about making the world a better ...
https://www.youtube.com/user/oceanworshipper,https://www.youtube.com/user/oceanworshipper,15.0,60.0,20461.0,15768000.0,https://www.youtube.com/user/oceanworshipper,nCKcW9wbMlM,2046.0,Advanced Alignment and Adjustments is part of ...,2018-07-05,...,381.0,Published,Donate to EcoKarma,237924.0,2008-02-26,3070.0,0.0,2.0,288.0,Blissology is about making the world a better ...
https://www.youtube.com/user/oceanworshipper,https://www.youtube.com/user/oceanworshipper,17.0,159.0,1602.0,31536000.0,https://www.youtube.com/user/oceanworshipper,KMGAlkw-MbA,160.0,Nature is the ultimate spiritual portal. When ...,2018-01-06,...,1120.0,Published,Donate to EcoKarma,237924.0,2008-02-26,3070.0,0.0,2.0,288.0,Blissology is about making the world a better ...
https://www.youtube.com/user/oceanworshipper,https://www.youtube.com/user/oceanworshipper,16.0,60.0,2312.0,31536000.0,https://www.youtube.com/user/oceanworshipper,HHudryLLmbk,231.0,The practice of yoga is a powerful tool that a...,2018-01-11,...,1193.0,Published,Donate to EcoKarma,237924.0,2008-02-26,3070.0,0.0,2.0,288.0,Blissology is about making the world a better ...
https://www.youtube.com/user/oceanworshipper,https://www.youtube.com/user/oceanworshipper,19.0,60.0,8392.0,31536000.0,https://www.youtube.com/user/oceanworshipper,AdoqTzZwljk,839.0,Aloha everyone! This January 22nd - February 4...,2017-10-16,...,529.0,Published,Donate to EcoKarma,237924.0,2008-02-26,3070.0,0.0,2.0,288.0,Blissology is about making the world a better ...


In [177]:
video_channel_df.shape

(10492, 22)

In [179]:
video_channel_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10492 entries, https://www.youtube.com/user/oceanworshipper to https://www.youtube.com/channel/UC8FPJGVe866y909klT_4AJA
Data columns (total 22 columns):
channel_URL_page        10271 non-null object
video_order_num         10271 non-null float64
duration                10271 non-null float64
views_video             10271 non-null float64
video_age_min           10271 non-null float64
channel_URL_info        10271 non-null object
video_URL_ext           10271 non-null object
views_DUPL              10271 non-null float64
video_desc              10271 non-null object
video_published_date    10271 non-null datetime64[ns]
likes                   10271 non-null float64
dislikes                10271 non-null float64
video_desc_length       10271 non-null float64
delivery_method         10271 non-null object
channel_name            8383 non-null object
total_views_channel     8383 non-null float64
channel_created_date    8383 non-null datetime64[ns

In [180]:
video_channel_df = video_channel_df[video_channel_df['channel_name'].notna()]

In [181]:
video_channel_df = video_channel_df[video_channel_df['video_URL_ext'].notna()]

In [186]:
video_channel_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8162 entries, https://www.youtube.com/user/oceanworshipper to https://www.youtube.com/user/yogayak
Data columns (total 22 columns):
channel_URL_page        8162 non-null object
video_order_num         8162 non-null float64
duration                8162 non-null float64
views_video             8162 non-null float64
video_age_min           8162 non-null float64
channel_URL_info        8162 non-null object
video_URL_ext           8162 non-null object
views_DUPL              8162 non-null float64
video_desc              8162 non-null object
video_published_date    8162 non-null datetime64[ns]
likes                   8162 non-null float64
dislikes                8162 non-null float64
video_desc_length       8162 non-null float64
delivery_method         8162 non-null object
channel_name            8162 non-null object
total_views_channel     8162 non-null float64
channel_created_date    8162 non-null datetime64[ns]
subscriber_count        8162 non-

In [184]:
video_channel_df[video_channel_df['channel_desc_length'].isna()] 

,channel_URL_page,video_order_num,duration,views_video,video_age_min,channel_URL_info,video_URL_ext,views_DUPL,video_desc,video_published_date,...,video_desc_length,delivery_method,channel_name,total_views_channel,channel_created_date,subscriber_count,n_featured_channels,n_promo_links,channel_desc_length,channel_desc
channel_URL,,,,,,,,,,,,,,,,,,,,,
https://www.youtube.com/channel/UC9-u2Db-Ltl0LEltmKGj22Q,https://www.youtube.com/channel/UC9-u2Db-Ltl0L...,28.0,273.0,22162.0,31536000.0,https://www.youtube.com/channel/UC9-u2Db-Ltl0L...,f0CHV4EZ81M,2217.0,,2017-10-07,...,0.0,Published,,155405.0,2012-03-15,2040.0,0.0,0.0,NaN,
https://www.youtube.com/channel/UC9-u2Db-Ltl0LEltmKGj22Q,https://www.youtube.com/channel/UC9-u2Db-Ltl0L...,6.0,167.0,14111.0,475200.0,https://www.youtube.com/channel/UC9-u2Db-Ltl0L...,DGw-ZY6OEaI,141.0,By Jose Calarco,2019-05-06,...,15.0,Published,,155405.0,2012-03-15,2040.0,0.0,0.0,NaN,
https://www.youtube.com/channel/UC9-u2Db-Ltl0LEltmKGj22Q,https://www.youtube.com/channel/UC9-u2Db-Ltl0L...,25.0,13.0,1421.0,15768000.0,https://www.youtube.com/channel/UC9-u2Db-Ltl0L...,dUqNuRK2qEU,142.0,"To learn more, check out Ana and Jose's classe...",2018-05-11,...,87.0,Published,,155405.0,2012-03-15,2040.0,0.0,0.0,NaN,
https://www.youtube.com/channel/UC9-u2Db-Ltl0LEltmKGj22Q,https://www.youtube.com/channel/UC9-u2Db-Ltl0L...,26.0,16.0,661.0,15768000.0,https://www.youtube.com/channel/UC9-u2Db-Ltl0L...,yuCJ3phyGuw,66.0,"To learn more, check out Ana and Jose's classe...",2018-05-11,...,87.0,Published,,155405.0,2012-03-15,2040.0,0.0,0.0,NaN,
https://www.youtube.com/channel/UC9-u2Db-Ltl0LEltmKGj22Q,https://www.youtube.com/channel/UC9-u2Db-Ltl0L...,23.0,42.0,1481.0,15768000.0,https://www.youtube.com/channel/UC9-u2Db-Ltl0L...,NwxJbCtyvTI,148.0,"To learn more, check out Ana and Jose's classe...",2018-05-11,...,87.0,Published,,155405.0,2012-03-15,2040.0,0.0,0.0,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.youtube.com/user/yogayak,https://www.youtube.com/user/yogayak,13.0,43200.0,136991.0,0.0,https://www.youtube.com/user/yogayak,kYVXSQaF1n0,13759.0,Any stream sound lovers? The soothing sound o...,2020-03-26,...,3307.0,Published,,207880503.0,2006-06-15,659000.0,0.0,0.0,NaN,
https://www.youtube.com/user/yogayak,https://www.youtube.com/user/yogayak,2.0,43200.0,65901.0,1440.0,https://www.youtube.com/user/yogayak,h6BlCAs6LgU,6769.0,A pleasant rushing stream sound. A fast movin...,2020-04-07,...,3162.0,Published,,207880503.0,2006-06-15,659000.0,0.0,0.0,NaN,
https://www.youtube.com/user/yogayak,https://www.youtube.com/user/yogayak,8.0,43200.0,150321.0,0.0,https://www.youtube.com/user/yogayak,N4SYOEexnxc,15145.0,"This is very close to white noise, yet it's un...",2020-03-31,...,3275.0,Published,,207880503.0,2006-06-15,659000.0,0.0,0.0,NaN,


In [188]:
video_channel_df['channel_desc_length'].fillna(value=0, inplace=True) #= video_channel_df[video_channel_df['channel_desc_length'].isna()] = 0

In [189]:
video_channel_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8162 entries, https://www.youtube.com/user/oceanworshipper to https://www.youtube.com/user/yogayak
Data columns (total 22 columns):
channel_URL_page        8162 non-null object
video_order_num         8162 non-null float64
duration                8162 non-null float64
views_video             8162 non-null float64
video_age_min           8162 non-null float64
channel_URL_info        8162 non-null object
video_URL_ext           8162 non-null object
views_DUPL              8162 non-null float64
video_desc              8162 non-null object
video_published_date    8162 non-null datetime64[ns]
likes                   8162 non-null float64
dislikes                8162 non-null float64
video_desc_length       8162 non-null float64
delivery_method         8162 non-null object
channel_name            8162 non-null object
total_views_channel     8162 non-null float64
channel_created_date    8162 non-null datetime64[ns]
subscriber_count        8162 non-

In [183]:
with open('videos_df_10271.pkl', 'wb') as picklefile:
    pickle.dump(videos_df, picklefile)

In [190]:
with open('video_channel_df_8162.pkl', 'wb') as picklefile:
    pickle.dump(video_channel_df, picklefile)

In [191]:
video_channel_df.describe()

,video_order_num,duration,views_video,video_age_min,views_DUPL,likes,dislikes,video_desc_length,total_views_channel,subscriber_count,n_featured_channels,n_promo_links,channel_desc_length
count,8162.000000,8162.000000,8.162000e+03,8.162000e+03,8.162000e+03,8162.000000,8162.000000,8162.000000,8.162000e+03,8.162000e+03,8162.000000,8162.000000,8162.000000
mean,13.304215,1033.716736,3.211180e+05,1.721729e+07,3.019288e+04,308.861186,14.096177,751.520951,9.512927e+06,6.133633e+04,1.123622,4.544964,432.257780
std,8.360356,2668.994682,5.522613e+06,2.625999e+07,5.492424e+05,3503.642629,264.907098,871.918550,5.342722e+07,3.396354e+05,2.471062,4.268182,353.594772
min,1.000000,2.000000,1.000000e+00,0.000000e+00,2.000000e+00,0.000000,0.000000,0.000000,1.820000e+02,0.000000e+00,0.000000,0.000000,0.000000
25%,6.000000,120.000000,1.935250e+03,4.320000e+04,1.760000e+02,4.000000,0.000000,153.000000,1.257220e+05,1.150000e+03,0.000000,0.000000,77.000000
50%,12.000000,398.000000,8.871000e+03,3.024000e+05,8.270000e+02,18.000000,0.000000,463.000000,4.041590e+05,4.040000e+03,0.000000,4.000000,353.000000
75%,20.000000,1140.000000,3.704550e+04,3.153600e+07,3.348250e+03,76.750000,2.000000,988.750000,2.542881e+06,2.770000e+04,1.000000,8.000000,789.000000
max,30.000000,43200.000000,2.748813e+08,4.336200e+08,2.750022e+07,216457.000000,13039.000000,4982.000000,7.463786e+08,6.870000e+06,10.000000,19.000000,1000.000000
